Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Runtime$\rightarrow$Factory reset runtime) and then **run all cells** (in the menubar, select Runtime$\rightarrow$Run all).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Fill out your name below in the `NAME` variable, leave the `COLLABORATORS` variable empty.

In [39]:
NAME = "Payal Mantri"
COLLABORATORS = ""

---

Install required dependency.

In [40]:
!pip install nose

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import required packages

In [41]:
import numpy as np

Download the required data.

In [42]:
!wget -c 'https://drive.google.com/uc?export=download&id=1rh6Rgjoyloyp2hPhTlOBD16yc7sc3gPy' -O data.zip
!unzip -o data.zip 

--2022-09-14 00:39:46--  https://drive.google.com/uc?export=download&id=1rh6Rgjoyloyp2hPhTlOBD16yc7sc3gPy
Resolving drive.google.com (drive.google.com)... 172.253.115.102, 172.253.115.113, 172.253.115.100, ...
Connecting to drive.google.com (drive.google.com)|172.253.115.102|:443... connected.
HTTP request sent, awaiting response... 416 Requested range not satisfiable

    The file is already fully retrieved; nothing to do.

Archive:  data.zip
  inflating: BLOSUM62.txt            
  inflating: Q2b_reference.txt       


Show the downloaded data

- `./BLOSUM62.txt`
- `./Q2b_reference.txt`


In [43]:
!ls *.txt

BLOSUM62.txt  Q2b_reference.txt


# Sequence Alignment

In bioinformatics, the problem of sequence alignment is reducible to a generalized form of edit distance. Sequence alignment is an instance of the weighted edit distance problem. Here, the cost of the elementary operations may vary according to the symbols involved. This generalization allows us to more accurately model processes in biological sequences that may lead to substitutions, insertions or deletions.

As we saw in lecture, we can store the costs of elementary operations in the form of a scoring matrix $\delta :$ ($\Sigma$ $\cup$ $\{-\})^{2}\rightarrow$ $\mathbb{R}$. The values of this scoring matrix depend on the biological context.

## BLOSUM62

The most widely used scoring matrix in amino acid sequence alignments is known as BLOSUM62. You can read more about BLOSUM here:

* https://www.sciencedirect.com/topics/biochemistry-genetics-and-molecular-biology/blosum
* https://en.wikipedia.org/wiki/BLOSUM

The BLOSUM.txt file that came in the HW1_supplement.zip contains the matrix. You can also download the BLOSUM62 matrix at https://www.ncbi.nlm.nih.gov/Class/FieldGuide/BLOSUM62.txt
 
Note: 
* The `read_blosum62` function is specifically tailored to read only the file above. You should not need to change this function. 
* The matrix uses the `*` character for gaps, but we change it here to the `-` character. 

In [44]:
def read_blosum62(path):
    """
    Reads in the ncbi's BLOSUM62.txt file and loads the scoring matrix
    into a dictionary.
    
    :param: path is the full path in the local filesystem at which the .txt file is located
    :return: a dictionary of dictionaries which will hold the cost of various amino acid
    substitutions as defined in BLOSUM62.
    """
    delta = {}
    with open(path, 'r') as f:
        lines = f.readlines()[6:]
        keys = lines[0].split()
        keys[-1] = '-'
        for i, line in enumerate(lines[1:]):
            delta[keys[i]] = {k : int(v) for (k,v) in zip(keys, line.split()[1:])}  
    return delta

blosum = read_blosum62('./BLOSUM62.txt')

## Question 1a - Global Alignment - 10 points

Complete the function `global_align` defined below that uses the Needleman-Wunsch algorithm for global sequence alignment. The function should take two strings, and a scoring function, and compute the score of the optimal global alignment. It should return a tuple `(best score, optimal alignment)`.

We have provided a function `traceback_global` that takes the two strings `v` and `w`, and a list of lists called
`pointers`. This list is what you will use to record backpointers while running your sequence alignment algorithm.
Each `pointers[i][j]` will be one of `UP`, `LEFT`, `TOPLEFT` or `ORIGIN`. The `traceback_global` function will insert gaps into the two strings where necessary, according to the backpointer table you pass it, and return the alignment to you. 

*Hint: Feel free to define helper functions for this and future questions on this notebook.*

In [45]:
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)

def traceback_global(v, w, pointers):
    i,j = len(v), len(w)
    new_v = []
    new_w = []
    while True:
        di, dj = pointers[i][j]
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
        i, j = i + di, j + dj
        if (i <= 0 and j <= 0):
            break
    return ''.join(new_v[::-1])+'\n'+''.join(new_w[::-1])
    


def global_align(v, w, delta):
    """
    Returns the score of the maximum scoring alignment of the strings v and w, as well as the actual alignment as 
    computed by traceback_global. 
    
    :param: v
    :param: w
    :param: delta
    """
    M = [[0 for j in range(len(w)+1)] for i in range(len(v)+1)]
    pointers = [[ORIGIN for j in range(len(w)+1)] for i in range(len(v)+1)]
    score, alignment = None, None
    # YOUR CODE HERE
    arrow = ORIGIN;

    for row  in range(len(v)+1):
      for column in range(len(w)+1):
        if row==0 and column==0:
           score=0;
        elif row == 0:
          #We're on the first row
          #but NOT in the corner
          
          #Get the score of the previous cell (to the left) in the matrix and add the gap penalty\
          previous_score = M[row][column - 1]
          # we are using column-1 to get index of sequence because table has one xtra roew abnd column
          score = M[row][column - 1] + delta['-'][w[column-1]]
          arrow = LEFT
        elif column == 0:
            #We're on the first column but not in the first row
         #  previous_score = M[row -1,column]
      
          score = M[row-1][column] + delta[v[row-1]]['-']
          arrow = UP
        else: 
          #We're in a 'middle' cell of the alignment
          
          #Calculate the scores for coming from above,
          #from the left, (representing an insertion into seq1)
          cell_to_the_left = M[row][column-1]
          from_left_score = M[row][column-1] + delta['-'][w[column-1]]
            
          #or from above (representing an insertion into seq2)
          above_cell = M[row-1][column]
          from_above_score = M[row-1][column] +  delta[v[row-1]]['-']
          
          #diagonal cell, representing a substitution (e.g. A --> T)
          diagonal_left_cell = M[row-1][column-1]
          

          #NOTE: since the table has an extra row and column (the blank ones), 
          #when indexing back to the sequence we want row -1 and col - 1.
          #since row 1 represents character 0 of the sequence.

          # delta[v[row-1]][w[column-1]] = gives match or mismatch penalty
          diagonal_left_cell_score = diagonal_left_cell + delta[v[row-1]][w[column-1]]
          
          score = max([from_left_score,from_above_score,diagonal_left_cell_score]) 
          #make note of which cell was the max in the traceback array 
          #using Unicode arrows
          if score == from_left_score:
              arrow = LEFT
          elif score == from_above_score:
              arrow = UP
          elif score == diagonal_left_cell_score:
              arrow = TOPLEFT
        pointers[row][column] = arrow
        M[row][column] = score
    
    alignment = traceback_global(v,w, pointers)
    return score, alignment

In [46]:
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
#print(delta)
global_align("TAGATA", "GTAGGCTTAAGGTTA", delta)

(-3, '-TAG----A---T-A\nGTAGGCTTAAGGTTA')

https://colab.research.google.com/github/zaneveld/full_spectrum_bioinformatics/blob/master/content/08_phylogenetic_trees/needleman_wunsch_alignment.ipynb#scrollTo=ZtuzMYw60DIS

---



## Question 1b -- Global Alignment, Protein Data Bank -- 5 points

https://www.rcsb.org/ provides a database of most known proteins. The search functionality allows us to find information on proteins via their *PDB ID* (https://www.rcsb.org/pdb/staticHelp.do?p=help/advancedsearch/pdbIDs.html). You can get the exact amino acid sequence of chains in a protein by searching via the PDB ID, then under the *Display Files* dropdown menu, clicking *FASTA Sequence*.

Use the function `global_align` with the BLOSUM62 scoring matrix to find the alignment score for the following pairs of amino acid sequences
* The A-chain of human insulin (#4F1C:A) and the A-chain of bovine insulin (#2ZP6:A). Set the variable `scoreA` below to this value.
* The B-chain of the above proteins. Set the variable `scoreB` below to this value.
* Set the `alignmentA` and `alignmentB` variables to the output of `traceback_global` function for both chains.

In [47]:
scoreA = 'GIVEQCCTSICSLYQLENYCN'
scoreB = 'GIVEQCCASVCSLYQLENYCN'

alignment= global_align(scoreA, scoreB, blosum)
print(alignment)
alignmentArray = alignment[1].split('\n')
if(len(alignmentArray)==2):
  alignmentA = alignmentArray[0]
  alignmentB = alignmentArray[1]
# print(alignmentA)
# print(alignmentB)

(115, 'GIVEQCCTSICSLYQLENYCN\nGIVEQCCASVCSLYQLENYCN')


In [48]:
import nose.tools as Test

# Student test cases (You may add more)
Test.assert_equal(global_align('LRRGEPVY', 'LEKGDTLYILVG', blosum), (5, 'LRRGEPVY----\nLEKGDTLYILVG'))
Test.assert_equal(global_align('ALYFFT', 'QCASVT', blosum), (-2, 'ALYFFT\nQCASVT'))
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
Test.assert_equal(global_align('GTTTTATAGC', 'TGGATCGAT', delta),(-3, 'GTTTTATAGC-\n-TGG-ATCGAT'))


## Question 2a - Fitting Alignment - 10 points

As we know, next generation sequencing doesn't produce a fully sequenced genome, but rather many short reads. Here, we use fitting alignment. 

Complete the `fitting_align` function defined below. The function should take a short read string, a reference genome string, and a scoring matrix, and return the score of the maximum scoring alignment of the short read and all substrings of the reference string, along with the actual fitting alignmnet as computed by the given function `traceback_fitting`. This function is similar to `traceback_global` but also requires you to pass in your dynamic programming table `M`, as well as the starting column for the backtrace `init_j`. 

** Note: In the case of ties with respect to determining the starting column for the backtrace, use the rightmost.**

In [49]:
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)

def traceback_fitting(v, w, M, init_j, pointers):
    i, j = len(v), init_j
    new_v = []
    new_w = []
    while True:
        di, dj = pointers[i][j]
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
        i, j = i + di, j + dj
        if (i <= 0):
            break
    return ''.join(new_v[::-1]) + '\n'+''.join(new_w[::-1])

def fitting_align(short, reference, delta):
    """
    Returns the score of the maximum scoring alignment of short and all 
    substrings of reference. 
    
    :param: short the shorter of the two strings we are trying to align    
    :param: reference the longer string among whose substrings we are doing global alignment
    :param: delta the scoring function for the alphabet of the two strings
    
    :returns: a tuple (score, alignment)
    """
    M = [[0 for j in range(len(reference)+1)] for i in range(len(short)+1)]     
    pointers = [[ORIGIN for j in range(len(reference)+1)] for i in range(len(short)+1)] 
    score = None
    arrow = ORIGIN
    init_j = 0
    
    for row  in range(len(short)+1):
      for column in range(len(reference)+1):
        if row==0 and column==0:
           score=0;
        elif row == 0:
          #We're on the first row
          #but NOT in the corner
          
          score = 0
          arrow = LEFT
        elif column == 0:
            #We're on the first column but not in the first row
         #  previous_score = M[row -1,column]
      
          score = M[row-1][column] + delta[short[row-1]]['-']
          arrow = UP
        else: 
          #We're in a 'middle' cell of the alignment
          
          #Calculate the scores for coming from above,
          #from the left, (representing an insertion into seq1)

          cell_to_the_left = M[row][column-1]
          from_left_score = M[row][column-1] + delta['-'][reference[column-1]]

          #or from above (representing an insertion into seq2)
        
          above_cell = M[row-1][column]
          from_above_score = M[row-1][column] +  delta[short[row-1]]['-']
          
          #diagonal cell, representing a substitution (e.g. A --> T)
          diagonal_left_cell = M[row-1][column-1]
          

          #NOTE: since the table has an extra row and column (the blank ones), 
          #when indexing back to the sequence we want row -1 and col - 1.
          #since row 1 represents character 0 of the sequence.

          # delta[v[row-1]][w[column-1]] = gives match or mismatch penalty
          diagonal_left_cell_score = diagonal_left_cell + delta[short[row-1]][reference[column-1]]
          
          score = max([from_left_score,from_above_score,diagonal_left_cell_score]) 
          #make note of which cell was the max in the traceback array 
          #using Unicode arrows
          if score == from_left_score:
              arrow = LEFT
          elif score == from_above_score:
              arrow = UP
          elif score == diagonal_left_cell_score:
              arrow = TOPLEFT
        pointers[row][column] = arrow
        M[row][column] = score

    numpyScoringMatrix = np.array(M)
    score = np.amax(numpyScoringMatrix[len(short)])
    init_j = np.argmax(numpyScoringMatrix[len(short)]) # get the value of column for which score is max
    #print(score)
    #print(np.argmax(numpyScoringMatrix[len(short)]))
    alignment = traceback_fitting(short,reference,M, init_j,pointers)
    return score, alignment

In [50]:
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
    
fitting_align("TAGATA", "GTAGGCTTAAGGTTA", delta)

(2, 'TAGA-T-A\nTAGGCTTA')

In [51]:
import nose.tools as Test

# Define scoring matrix
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
delta['-']['-'] = -1

# Student test cases (You can add more)
Test.assert_equal(fitting_align('ACTGCT', 'ACTGTCGTACGTGTACGTGCTATTACGATTCGGATGCATTGTGCATTTGGGCGATCTTATTCTTATC', delta),\
                  (5, 'AC-TGCT\nACGTGCT'))
Test.assert_equal(fitting_align('PPPVP','MHHHHHHSSPIDPPGKPVPLNITRHTVTLKWAKPEYTGGFKITSYIVEKRDLPNGRWLKANFSNILENEFTVSGLTEDAA\
YEFRVIAKNAAGAISPPSEPSDAITCRDDVEA', blosum), (24, 'PP--PVP\nPPGKPVP'))


## Question 2b - Fitting Alignment - 5 points

After writing the function use it to align the reference sequence `./Q2b_reference.txt` and the short read provided below. The reference is a partial version of the human Y-chromosome reference found at ftp://ftp.ncbi.nih.gov/genomes/H_sapiens/CHR_Y/. 

Set the variable `score_fitting` to the fitting alignment score, and set the `alignment_fitting` to the output of the `traceback_fitting` function. You can use the scoring matrix defined below. 

** Note: The reference sequence is very large, and so the fitting alignment will take a few minutes. Make sure you use dynamic programming to implement your alignment functions, or the cell will never finish. **

In [52]:
short = 'AGAAGTGAGTTTTGGATAGTAAAATAAGTTTCGAACTCTGGCACCTTTCAATTTTGTCGCACTCTCCTTGTTTTTGACAATGCAATCATATGCTTCTGCT'
keys = ['A', 'C', 'T', 'G', '-']
delta_fitting = {}
for i in range(len(keys)):
    delta_fitting[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
delta_fitting['-']['-'] = -1

# Read the reference sequence, and call your fittinga alignment function

with open('./Q2b_reference.txt') as f:
    reference = f.readline()
    
fitting_response = fitting_align(short, reference, delta_fitting)
score_fitting = fitting_response[0]
alignment_fitting = fitting_response[1]




**Solution:**

score = 33 

**Alignment**

AGAAG-T-GAGTTTTGGATAGTAAAATAAGTTT-CGAAC-TCTGG---CACC-TT-T-CAATTTTGTCGC-ACTCTCCTTG-TTTT-TGACAA--TGCAATCATAT-GCTT-C-TGC-T
AGATGATTG-GTTTTAGATATTT---TAAGTTTTCAAACATATGGATTCAGTGTTATACA-TTTC-TCTCTACTC-CCT-GCTTTTCTGT-AACCTGCAA--ATATTGATAGCGTGCAT


## Question 2c - 5 points

Find the name of the gene the short read in the question above originated from. Save your answer in variable `ANSWER` below. 

*Hint : There is an online tool that will do this for you*

In [54]:
# YOUR CODE HERE
#Homo sapiens chromosome Y, GRCh38.p14 Primary Assembly
#Homo sapiens sex determining region Y (SRY), mRNA
ANSWER = 'Homo sapiens chromosome Y, GRCh38.p14 Primary Assembly'

Used this link https://blast.ncbi.nlm.nih.gov/Blast.cgi

## Question 3 - Local Alignment - 15 points

Finally, we use local alignment for things like aligning functional units of proteins called domains. 

Use the Smith-Waterman algorithm to complete the function `local_align` defined below that takes two strings `v` and `w` and a scoring matrix, and returns the maximum scoring global alignment across all substrings of `v` and `w`, as well as the alignment string produced by `traceback_local`. As with the above functions, you must record backpointers. This function is very similar to `traceback_fitting`, except it also takes the starting cell for the back trace in the form of coordinates `init_i, init_j`. 

** Note: In the case of ties with respect to determining the starting column for the backtrace, use the bottom-right most cell.**

In [56]:
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)

def traceback_local(v, w, M, init_i, init_j, pointers):
    i,j = init_i, init_j
    new_v = []
    new_w = []
    while True:
        di, dj = pointers[i][j]
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
        i, j = i + di, j + dj
        if (M[i][j] == 0):
            break
    return ''.join(new_v[::-1]) + '\n'+''.join(new_w[::-1])

def local_align(v, w, delta):
    """
    Returns the score of the maximum scoring alignment of all possible substrings of v and w. 
    
    :param: v
    :param: w
    :param: delta
    """
    M = [[0 for j in range(len(w)+1)] for i in range(len(v)+1)]        
    pointers = [[ORIGIN for j in range(len(w)+1)] for i in range(len(v)+1)] 
    score = None
    init_i, init_j = 0,0
    # YOUR CODE HERE
    arrow = ORIGIN;
    best_score=0;
    best_location =(0,0);

    for row  in range(1, len(v)+1):
      for column in range(1, len(w)+1):
        #Calculate the scores for coming from above,
        #from the left, (representing an insertion into seq1)
        from_left_score = M[row][column-1] + delta['-'][w[column-1]]
          
        #or from above (representing an insertion into seq2)
        from_above_score = M[row-1][column] +  delta[v[row-1]]['-']
        
        #diagonal cell, representing a substitution (e.g. A --> T)
        diagonal_left_cell = M[row-1][column-1]
        

        #NOTE: since the table has an extra row and column (the blank ones), 
        #when indexing back to the sequence we want row -1 and col - 1.
        #since row 1 represents character 0 of the sequence.

        # delta[v[row-1]][w[column-1]] = gives match or mismatch penalty
        diagonal_left_cell_score = diagonal_left_cell + delta[v[row-1]][w[column-1]]
        
        score = max([from_left_score,from_above_score,diagonal_left_cell_score, 0]) 
        #make note of which cell was the max in the traceback array 
        #using Unicode arrows
        if score == from_left_score:
            arrow = LEFT
        elif score == from_above_score:
            arrow = UP
        elif score == diagonal_left_cell_score:
            arrow = TOPLEFT
        if(score>=best_score):
          best_score = score
          best_location = (row, column)
        pointers[row][column] = arrow
        M[row][column] = score

    init_i = best_location[0]
    init_j= best_location[1]
    alignment = traceback_local(v, w, M, init_i, init_j , pointers)
    return best_score, alignment 

In [57]:
keys = ['A', 'C', 'T', 'G', '-']
delta = {}
for i in range(len(keys)):
    delta[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
    
local_align("TAGATA", "GTAGGCTTAAGGTTA", delta)

(3, 'TAG\nTAG')

Use your `local_align` function to find the common domain between the two amino acid sequences given below. 
* Set the `score_local` variable to the output of your function. 
* Set the `alignment_local` variable to the output of the `traceback_local` function. 

In [58]:
LTK_MOUSE = "MGCSHRLLLWLGAAGTILCSNSEFQTPFLTPSLLPVLVLNSQEQKVTPTP\
SKLEPASLPNPLGTRGPWVFNTCGASGRSGPTQTQCDGAYTGSSVMVTVG\
AAGPLKGVQLWRVPDTGQYLISAYGAAGGKGAQNHLSRAHGIFLSAVFFL\
RRGEPVYILVGQQGQDACPGGSPESQLVCLGESGEHATTYGTERIPGWRR\
WAGGGGGGGGATSIFRLRAGEPEPLLVAAGGGGRSYRRRPDRGRTQAVPE\
RLETRAAAPGSGGRGGAAGGGSGWTSRAHSPQAGRSPREGAEGGEGCAEA\
WAALRWAAAGGFGGGGGACAAGGGGGGYRGGDTSESDLLWADGEDGTSFV\
HPSGELYLQPLAVTEGHGEVEIRKHPNCSHCPFKDCQWQAELWTAECTCP\
EGTELAVDNVTCMDLPTTASPLILMGAVVAALALSLLMMCAVLILVNQKC\
QGLWGTRLPGPELELSKLRSSAIRTAPNPYYCQVGLSPAQPWPLPPGLTE\
VSPANVTLLRALGHGAFGEVYEGLVTGLPGDSSPLPVAIKTLPELCSHQD\
ELDFLMEALIISKFSHQNIVRCVGLSFRSAPRLILLELMSGGDMKSFLRH\
SRPHPGQLAPLTMQDLLQLAQDIAQGCHYLEENHFIHRDIAARNCLLSCS\
GASRVAKIGDFGMARDIYQASYYRKGGRTLLPVKWMPPEALLEGLFTSKT\
DSWSFGVLLWEIFSLGYMPYPGHTNQEVLDFIATGNRMDPPRNCPGPVYR\
IMTQCWQHQPELRPDFGSILERIQYCTQDPDVLNSPLPVEPGPILEEEEA\
SRLGNRSLEGLRSPKPLELSSQNLKSWGGGLLGSWLPSGLKTLKPRCLQP\
QNIWNPTYGSWTPRGPQGEDTGIEHCNGSSSSSIPGIQ"

ATK_MOUSE = "MGAAGFLWLLPPLLLAAASYSGAATDQRAGSPASGPPLQPREPLSYSRLQ\
RKSLAVDFVVPSLFRVYARDLLLPQPRSPSEPEAGGLEARGSLALDCEPL\
LRLLGPLPGISWADGASSPSPEAGPTLSRVLKGGSVRKLRRAKQLVLELG\
EETILEGCIGPPEEVAAVGILQFNLSELFSWWILHGEGRLRIRLMPEKKA\
SEVGREGRLSSAIRASQPRLLFQIFGTGHSSMESPSETPSPPGTFMWNLT\
WTMKDSFPFLSHRSRYGLECSFDFPCELEYSPPLHNHGNQSWSWRHVPSE\
EASRMNLLDGPEAEHSQEMPRGSFLLLNTSADSKHTILSPWMRSSSDHCT\
LAVSVHRHLQPSGRYVAQLLPHNEAGREILLVPTPGKHGWTVLQGRVGRP\
ANPFRVALEYISSGNRSLSAVDFFALKNCSEGTSPGSKMALQSSFTCWNG\
TVLQLGQACDFHQDCAQGEDEGQLCSKLPAGFYCNFENGFCGWTQSPLSP\
HMPRWQVRTLRDAHSQGHQGRALLLSTTDILASEGATVTSATFPAPMKNS\
PCELRMSWLIRGVLRGNVSLVLVENKTGKEQSRTVWHVATDEGLSLWQHT\
VLSLLDVTDRFWLQIVTWWGPGSRATVGFDNISISLDCYLTISGEEKMSL\
NSVPKSRNLFEKNPNKESKSWANISGPTPIFDPTVHWLFTTCGASGPHGP\
TQAQCNNAYQNSNLSVVVGSEGPLKGVQIWKVPATDTYSISGYGAAGGKG\
GKNTMMRSHGVSVLGIFNLEKGDTLYILVGQQGEDACPRANQLIQKVCVG\
ENNVIEEEIRVNRSVHEWAGGGGGGGGATYVFKMKDGVPVPLIIAAGGGG\
RAYGAKTETFHPERLESNSSVLGLNGNSGAAGGGGGWNDNTSLLWAGKSL\
LEGAAGGHSCPQAMKKWGWETRGGFGGGGGGCSSGGGGGGYIGGNAASNN\
DPEMDGEDGVSFISPLGILYTPALKVMEGHGEVNIKHYLNCSHCEVDECH\
MDPESHKVICFCDHGTVLADDGVSCIVSPTPEPHLPLSLILSVVTSALVA\
ALVLAFSGIMIVYRRKHQELQAMQMELQSPEYKLSKLRTSTIMTDYNPNY\
CFAGKTSSISDLKEVPRKNITLIRGLGHGAFGEVYEGQVSGMPNDPSPLQ\
VAVKTLPEVCSEQDELDFLMEALIISKFNHQNIVRCIGVSLQALPRFILL\
ELMAGGDLKSFLRETRPRPNQPTSLAMLDLLHVARDIACGCQYLEENHFI\
HRDIAARNCLLTCPGAGRIAKIGDFGMARDIYRASYYRKGGCAMLPVKWM\
PPEAFMEGIFTSKTDTWSFGVLLWEIFSLGYMPYPSKSNQEVLEFVTSGG\
RMDPPKNCPGPVYRIMTQCWQHQPEDRPNFAIILERIEYCTQDPDVINTA\
LPIEYGPVVEEEEKVPMRPKDPEGMPPLLVSPQPAKHEEASAAPQPAALT\
APGPSVKKPPGAGAGAGAGAGAGPVPRGAADRGHVNMAFSQPNPPPELHK\
GPGSRNKPTSLWNPTYGSWFTEKPAKKTHPPPGAEPQARAGAAEGGWTGP\
GAGPRRAEAALLLEPSALSATMKEVPLFRLRHFPCGNVNYGYQQQGLPLE\
ATAAPGDTMLKSKNKVTQPGP"

score_local, alignment_local = None, None
# YOUR CODE HERE
result=local_align(LTK_MOUSE, ATK_MOUSE, blosum)

score_local =  result[0]
alignment_local = result[1]

In [59]:
import nose.tools as Test
# Student test cases (You may add more)
keys = ['A', 'C', 'T', 'G', '-']
delta_local = {}
for i in range(len(keys)):
    delta_local[keys[i]] = {k : v for (k,v) in zip(keys, [1 if keys[i] == keys[j]  else -1 for j in range(len(keys))])}
delta_local['-']['-'] = -1
Test.assert_equal(local_align("TACGTGACGTCTATCAT", "TTGTGCATGTATCTGAC", delta_local), (6, 'GTGACGTCTATCAT\nGTG-CATGTATC-T'))
Test.assert_equal(local_align("ACTGATTTCGATGCTGTACGTGACGTACGTTTATTCTATCAT", "TCTGACTGTGACTATACTATTGTGCATGTATCTGACTAGCTAG", delta_local),\
                  (13, 'ACTGATTTCGATGC-TGTA-CGTGACGTACGTTTA\nACT-ATT--G-TGCATGTATC-TGAC-TA-GCT-A'))
